In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }
    
    response = requests.put(url, headers=headers, json=data)
    
    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
from flask import Flask, request, abort, send_file
import logging
import os
from google.genai import types
from io import BytesIO
import uuid
import time
import threading
import base64
from PIL import Image

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    PushMessageRequest,
    TextMessage,
    VideoMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    ImageMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_images"
VIDEO_DIR = "/content/generated_videos"
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

# 下載 LINE 圖片的函數
def download_line_image(message_id):
    """從 LINE 下載使用者上傳的圖片"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        image_content = line_bot_blob_api.get_message_content(message_id)
        
        image_filename = f"{uuid.uuid4()}.jpg"
        image_path = os.path.join(UPLOAD_DIR, image_filename)
        
        with open(image_path, 'wb') as f:
            f.write(image_content)
        
        return image_path

# 分析圖片產生 prompt 的函數
def generate_video_prompt(image_path):
    """使用 Gemini 分析圖片並產生完整的影片 prompt"""
    image = Image.open(image_path)
    
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=[
            "請根據這張圖片，用英文產生一個詳細的描述（約 50 字），包括：1) 圖片的主要物體、顏色、場景 2) 建議的動作或鏡頭移動。格式：先描述畫面，然後描述動作。只輸出英文 prompt，不要其他說明。",
            image
        ]
    )
    
    return response.text.strip()

# 從圖片產生影片的函數
def generate_video_from_image(image_path, prompt):
    """使用 Veo 3.1 從圖片產生影片"""
    # 使用 gemini-2.5-flash-image 根據 prompt 產生圖片
    image_response = client.models.generate_content(
        model="gemini-2.5-flash-image",
        contents=prompt,
        config={"response_modalities": ['IMAGE']}
    )
    
    # 使用 Veo 3.1 產生影片
    operation = client.models.generate_videos(
        model="veo-3.1-generate-preview",
        prompt=prompt,
        image=image_response.parts[0].as_image(),
    )
    
    # 輪詢等待影片產生完成
    while not operation.done:
        print("等待影片產生中...")
        time.sleep(10)
        operation = client.operations.get(operation)
    
    print(f"操作完成，狀態：{operation}")
    
    # 檢查是否有錯誤
    if hasattr(operation, 'error') and operation.error:
        raise Exception(f"影片產生失敗：{operation.error}")
    
    # 檢查 response 是否存在
    if not hasattr(operation, 'response') or not operation.response:
        raise Exception(f"影片產生完成但沒有回傳結果。Operation 資訊：{operation}")
    
    # 檢查是否有產生的影片
    if not hasattr(operation.response, 'generated_videos') or not operation.response.generated_videos:
        raise Exception(f"沒有產生影片。Response 內容：{operation.response}")
    
    # 下載影片
    video = operation.response.generated_videos[0]
    client.files.download(file=video.video)
    
    # 儲存影片
    video_filename = f"{uuid.uuid4()}.mp4"
    video_path = os.path.join(VIDEO_DIR, video_filename)
    video.video.save(video_path)
    
    return video_filename

# 背景處理影片產生的函數
def process_video_generation(user_id, image_path):
    """在背景執行影片產生並推送給使用者"""
    try:
        # 產生影片 prompt
        prompt = generate_video_prompt(image_path)
        print(f"產生的 prompt: {prompt}")
        
        # 產生影片
        video_filename = generate_video_from_image(image_path, prompt)
        
        # 取得影片 URL
        ngrok_url = tunnel.public_url
        video_url = f"{ngrok_url}/videos/{video_filename}"
        
        # 使用 push_message 傳送影片給使用者
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.push_message(
                PushMessageRequest(
                    to=user_id,
                    messages=[
                        TextMessage(text=f"🎬 影片產生完成！\nPrompt: {prompt}"),
                        VideoMessage(
                            original_content_url=video_url,
                            preview_image_url=f"{ngrok_url}/uploads/{os.path.basename(image_path)}"
                        )
                    ]
                )
            )
    except Exception as e:
        # 發生錯誤時通知使用者
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.push_message(
                PushMessageRequest(
                    to=user_id,
                    messages=[TextMessage(text=f"❌ 影片產生失敗：{str(e)}")]
                )
            )

@app.route("/videos/<filename>")
def serve_video(filename):
    """提供影片存取的路由"""
    video_path = os.path.join(VIDEO_DIR, filename)
    if os.path.exists(video_path):
        return send_file(video_path, mimetype='video/mp4')
    else:
        abort(404)

@app.route("/uploads/<filename>")
def serve_upload(filename):
    """提供上傳圖片存取的路由（作為影片預覽圖）"""
    image_path = os.path.join(UPLOAD_DIR, filename)
    if os.path.exists(image_path):
        return send_file(image_path, mimetype='image/jpeg')
    else:
        abort(404)

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_image_message(event):
    """處理使用者上傳的圖片，產生影片"""
    user_id = event.source.user_id
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        try:
            # 下載圖片
            image_path = download_line_image(event.message.id)
            
            # 先回覆使用者正在處理中
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="🎬 收到圖片！正在產生影片中，請稍候約 2-5 分鐘...")]
                )
            )
            
            # 在背景執行影片產生
            thread = threading.Thread(
                target=process_video_generation,
                args=(user_id, image_path)
            )
            thread.start()
            
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 處理圖片時發生錯誤：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)